In [1]:
import os
import numpy as np 
import pandas as pd 
import tensorflow
import random
import cv2
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Dense
from keras.layers import Flatten
from tensorflow.keras.optimizers import SGD
import warnings
from keras import backend as K
from sklearn.model_selection import train_test_split
from PIL import Image
from keras.callbacks import EarlyStopping
from keras.layers import Dense, Activation, Dropout, Flatten,Conv2D, MaxPooling2D
from tensorflow.keras.layers import BatchNormalization
from sklearn.metrics import confusion_matrix
from keras.models import Model
from keras.layers import Dense, Conv2D, MaxPool2D , Flatten
from keras.models import load_model

In [4]:
def load_images_from_folder(folder):
	images = []
	dirs = os.listdir(folder)

	for filename in dirs:
		if os.path.isfile(folder+filename):
			im = Image.open(folder+filename)
			imResize = im.resize((224,224), Image.ANTIALIAS)
			imResize = np.array(imResize)
			if imResize is not None:
				images.append(imResize)
	return images


def prepareData(parentPath):
	
	# Make a list of all the 0 label and 1 label images for train, val, and test sets

	all0_path = list()
	

	all1_path = list()
	

	all0_path.append(parentPath+'/0/')
	all1_path.append(parentPath+'/1/')


	# Read images into respective lists
	

	allX = list()
	allY = list()
	xTrain = list()
	yTrain = list()
	xVal = list()
	yVal = list()
	xTest = list()
	yTest = list()

	print('Class 0, reading started..\n\n')
	

	tempImgs = list()
	tempImgs = load_images_from_folder(all0_path[0])
	for i in range(len(tempImgs)):
		allX.append(tempImgs[i])
		allY.append(0)

	# Prepare all data for 1 class
	print('Class 1, reading started..\n\n')
	tempImgs = list()
	tempImgs = load_images_from_folder(all1_path[0])
	for i in range(len(tempImgs)):
		allX.append(tempImgs[i])
		allY.append(1)

	xTrain,xTest,yTrain,yTest = train_test_split(allX,allY, test_size=0.20, random_state=23, shuffle = True)

	xpTrain,xVal,ypTrain,yVal = train_test_split(xTrain,yTrain, test_size=0.10, random_state=23, shuffle = True)

	return xpTrain, ypTrain, xVal, yVal, xTest, yTest

In [5]:
parentPath = 'DFU-Dataset2'
xTrain, yTrain, xVal, yVal, xTest, yTest = prepareData(parentPath)
xTrain = np.array(xTrain)
yTrain = np.array(yTrain)
xVal = np.array(xVal)
yVal = np.array(yVal)
xTest = np.array(xTest)
yTest = np.array(yTest)

# Prepare the data
img_width, img_height = 224, 224

if K.image_data_format() == 'channels_first': 
  input_shape = (3, img_width, img_height) 
else:
  input_shape = (img_width, img_height, 3) 

# normalize inputs from 0-255 to 0-1
xTrain = xTrain / 255
xVal = xVal / 255
xTest = xTest / 255

Class 0, reading started..


Class 1, reading started..




In [6]:
from keras.applications.inception_v3 import InceptionV3
def InceptionV3_transfer_actual(input_shape):
  res = InceptionV3(weights=None, include_top=False, input_shape=input_shape)
  for layers in res.layers:
    layers.trainable = True

  model = Sequential()
  model.add(res)
  
  model.add(Flatten())

  ## The following is the vanilla VGG16 architecture's FC layers
  model.add(Dense(units=4096,activation="relu"))
  model.add(Dense(units=4096,activation="relu"))
  model.add(Dense(units=1000,activation="relu"))
  model.add(Dense(units=1, activation="sigmoid"))

  opt = SGD(lr=0.001, momentum=0.9)
  model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])
  model.summary()
  return model

In [7]:
import tensorflow as tf
from sklearn.metrics import roc_auc_score

def auroc(y_true, y_pred):
  return tf.py_func(roc_auc_score, (y_true, y_pred), tf.double)

In [8]:
model1 = InceptionV3_transfer_actual(input_shape)
es = EarlyStopping(monitor='val_accuracy', mode='max', verbose=1, patience=200, restore_best_weights=True)
model1.fit(xTrain, yTrain, epochs=30, batch_size=10, validation_data=(xVal, yVal), verbose=1, callbacks = [es])

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 inception_v3 (Functional)   (None, 5, 5, 2048)        21802784  
                                                                 
 flatten (Flatten)           (None, 51200)             0         
                                                                 
 dense (Dense)               (None, 4096)              209719296 
                                                                 
 dense_1 (Dense)             (None, 4096)              16781312  
                                                                 
 dense_2 (Dense)             (None, 1000)              4097000   
                                                                 
 dense_3 (Dense)             (None, 1)                 1001      
                                                                 
Total params: 252,401,393
Trainable params: 252,366,961


/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


Epoch 1/30
122/122 [==============================] - 838s 7s/step - loss: 0.6078 - accuracy: 0.6650 - val_loss: 0.7617 - val_accuracy: 0.4338
Epoch 2/30
122/122 [==============================] - 819s 7s/step - loss: 0.4814 - accuracy: 0.7621 - val_loss: 0.6378 - val_accuracy: 0.5956
Epoch 3/30
122/122 [==============================] - 809s 7s/step - loss: 0.4603 - accuracy: 0.7926 - val_loss: 0.5093 - val_accuracy: 0.7647
Epoch 4/30
122/122 [==============================] - 796s 7s/step - loss: 0.4181 - accuracy: 0.8074 - val_loss: 0.4176 - val_accuracy: 0.8088
Epoch 5/30
122/122 [==============================] - 791s 6s/step - loss: 0.4087 - accuracy: 0.8148 - val_loss: 0.3421 - val_accuracy: 0.8603
Epoch 6/30
122/122 [==============================] - 786s 6s/step - loss: 0.3704 - accuracy: 0.8337 - val_loss: 0.5020 - val_accuracy: 0.7500
Epoch 7/30
122/122 [==============================] - 791s 6s/step - loss: 0.3795 - accuracy: 0.8379 - val_loss: 0.4953 - val_accuracy: 0.7941

KeyboardInterrupt: ignored

In [9]:
model1.save('inception_v3.h5')

In [10]:
model1 = InceptionV3_transfer_actual(input_shape)
model1.load_weights('inception_v3.h5') 

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 inception_v3 (Functional)   (None, 5, 5, 2048)        21802784  
                                                                 
 flatten_1 (Flatten)         (None, 51200)             0         
                                                                 
 dense_4 (Dense)             (None, 4096)              209719296 
                                                                 
 dense_5 (Dense)             (None, 4096)              16781312  
                                                                 
 dense_6 (Dense)             (None, 1000)              4097000   
                                                                 
 dense_7 (Dense)             (None, 1)                 1001      
                                                                 
Total params: 252,401,393
Trainable params: 252,366,96

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


In [11]:
testPreds1 = model1.predict(xTest)

In [12]:
predictedclas=[]
for i in testPreds1:
  if(i>0.5):
    predictedclas.append(1)
  else:
    predictedclas.append(0)
np.asarray(predictedclas)

array([1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0,
       0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1,
       1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1,
       1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1,
       1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1,
       1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0,
       1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1,
       1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1,
       0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1,
       0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0,
       1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0,
       1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1,
       0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1,
       1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0,

In [ ]:
yTest

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0,
       0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1,
       1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1,
       1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1,
       1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1,
       1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0,
       1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1,
       1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1,
       0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1,
       0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1,
       1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1,
       1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1,
       0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1,
       1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0,

In [13]:
ypred=np.asarray(predictedclas)
from sklearn.metrics import classification_report,confusion_matrix
from sklearn.metrics import accuracy_score
print(classification_report(yTest, ypred))
print('accuracy_score:')
print(accuracy_score(yTest, ypred))
confusion_matrix(yTest, ypred)

              precision    recall  f1-score   support

           0       0.75      0.87      0.81       119
           1       0.93      0.84      0.88       219

    accuracy                           0.86       338
   macro avg       0.84      0.86      0.85       338
weighted avg       0.86      0.86      0.86       338

accuracy_score:
0.8550295857988166


array([[104,  15],
       [ 34, 185]])